Chi merge is a simple algorithm that uses the chi-square statistic to discretize numeric attributes. 
It is a supervised, bottom-up data discretization method. It checks each pair of adjacent rows in order 
to determine if the class frequencies of the two intervals are significantly different. It tests the 
following hypothesis: that the two adjacent intervals are independent. If the hypothesis is confirmed the 
intervals are merged into a single interval, if not, they remain separated.

To understand the Chi Merge we will use the Iris Dataset obtained from university of California-Irvine machine leanring
repository. As a dataset to be discretized, we will perform data discretization for each of the 4 numeric attributes 
using chi merge method.

The stopping criteria will be max-interval = 6

Iris data
The data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant.

Attirbutes of the data
1. sepal length in cm 
2. sepal width in cm 
3. petal length in cm 
4. petal width in cm 
5. class: a)Iris Setosa b)Iris Versicolour c)Iris Virginica

In [1]:
#Import the libraries
import pandas as pd
import numpy as np
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Load the IRIS dataset
iris_df = pd.read_csv('iris.data',sep=',')

In [3]:
#Create the column headers
attributes = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
iris_df.columns = attributes

In [4]:
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [5]:
len(iris_df)

149

In [6]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 5 columns):
sepal_length    149 non-null float64
sepal_width     149 non-null float64
petal_length    149 non-null float64
petal_width     149 non-null float64
class           149 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [7]:
iris_df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,149.000000,149.000000,149.000000,149.000000
mean,5.848322,3.051007,3.774497,1.205369
std,0.828594,0.433499,1.759651,0.761292
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.400000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [8]:
# Function to merge the intervals with the lowest chi2 value and then recreate the list of intervals 
def  merge_build_new_intervals(chi2_list, intervals):
    
    new_intervals = []
    skip = False
    merged = False
    
    # Find the minimal Chi2
    min_chi2 = min(chi2_list) 
    
    # Find and Store the index for the minimum chi2
    for ctr, chi in enumerate(chi2_list):
        if chi == min_chi2:
            min_chi_index = ctr
            break
              
    for ctr in range(len(intervals)):
        #Skip one row to handle merging
        if skip:
            skip = False
            continue
        
        #Find the 2 rows with minimum chi2 and merge their intervals
        #Store the new set set of intervals
        if ctr == min_chi_index and not merged:
            t = intervals[ctr] + intervals[ctr+1]
            new_intervals.append([min(t), max(t)]) 
            skip = True
            merged = True
        else:
            new_intervals.append(intervals[ctr])
            
    return new_intervals

In [63]:
# Function to calculate the chi2 values for all the pair of intervals
def calculate_chi_square(data, intervals, attr, label):
    
    chi2 = []
    
    # Get all possible labels for 
    labels = sorted(set(data[label])) 
    
    # A helper function for padding the Counter()
    empty_count = {l: 0 for l in labels}
    n=0
    # Calcuate the chi2 values for every pair of intervals and store
    for i in range(len(intervals)-1):
        
        # List of samples in the first interval
        list_0 = data[data[attr].between(intervals[i][0], intervals[i][1])]
       
        # List of samples in the second interval
        list_1 = data[data[attr].between(intervals[i+1][0], intervals[i+1][1])]
        
        # Total no of sample in the two intervals
        N = len(list_0) + len(list_1)
    
        # No of samples in each class in the first interval
        A_0_arr = np.array([v for i, v in {**empty_count, **Counter(list_0[label])}.items()]) 
        
        # No of samples in each class in the second interval
        A_1_arr = np.array([v for i, v in {**empty_count, **Counter(list_1[label])}.items()])

        # No of samples in each class in the two intervals combined
        C = A_0_arr + A_1_arr 
       
        # Total no of samples in the first interval
        R_0 = sum(A_0_arr)
        # Total no of samples in the second interval
        R_1 = sum(A_1_arr)

        E_0 = (C*R_0)/N
        E_1 = (C*R_1)/N  
        print(E_0)
        print(E_1)

        # Calculate the chi2 for the 2 intervals
        chi2_ = (A_0_arr - E_0)**2/E_0 + (A_1_arr - E_1)**2/E_1
        chi2_ = np.nan_to_num(chi2_)
        #print(sum(chi2_))
        
        #Store the chi2 for the 2 intervals
        chi2.append(sum(chi2_)) 
     
    return chi2

In [61]:
# Function implementing the Chi merge Algorithm
def chimerge(data, attr, label, max_intervals):
    
    # Sort the distinct values for the passed attributes
    distinct_vals = sorted(set(data[attr]))
       
    # Initialize the intervals for each attribute
    intervals = [[distinct_vals[i], distinct_vals[i]] for i in range(len(distinct_vals))] 
   
    # Iterative keep merging rows with lowest chi2 values till the no of intervals remaining 
    # is greater than max_intervals
    while len(intervals) > max_intervals:
        
        # Find the chi value for each pair of attribute intervals
        chi = calculate_chi_square(data, intervals, attr, label)
        print(chi)
        #Merge the two intervals with the lowest chi2 and create a new list of intervals
        intervals = merge_build_new_intervals(chi, intervals)        
    
    n=0
    # Print the discretized intervals along with the attribute frquency in the interval
    for ctr in intervals:
        print('[', ctr[0], ',', ctr[1], ']','freq - ', len(data[attr][data[attr].between(ctr[0], ctr[1])]))

In [59]:
# Apply the Chi Merge to find the discretized intervals and frequencies for each attribute
for attr in ["sepal_length", "sepal_width", "petal_length", "petal_width"]:
    print('Interval for', attr)
    chimerge(data=iris_df, attr=attr, label='class', max_intervals=6)

Interval for sepal_length
[ 4.3 , 4.8 ] freq -  16
[ 4.9 , 5.4 ] freq -  35
[ 5.5 , 5.7 ] freq -  21
[ 5.8 , 6.2 ] freq -  26
[ 6.3 , 7.0 ] freq -  39
[ 7.1 , 7.9 ] freq -  12
Interval for sepal_width
[ 2.0 , 2.2 ] freq -  4
[ 2.3 , 2.4 ] freq -  7
[ 2.5 , 2.8 ] freq -  36
[ 2.9 , 2.9 ] freq -  10
[ 3.0 , 3.3 ] freq -  57
[ 3.4 , 4.4 ] freq -  35
Interval for petal_length
[ 1.0 , 1.9 ] freq -  49
[ 3.0 , 4.4 ] freq -  29
[ 4.5 , 4.7 ] freq -  16
[ 4.8 , 4.9 ] freq -  9
[ 5.0 , 5.1 ] freq -  12
[ 5.2 , 6.9 ] freq -  34
Interval for petal_width
[ 0.1 , 0.6 ] freq -  49
[ 1.0 , 1.3 ] freq -  28
[ 1.4 , 1.6 ] freq -  24
[ 1.7 , 1.7 ] freq -  2
[ 1.8 , 1.8 ] freq -  12
[ 1.9 , 2.5 ] freq -  34


In [12]:
# Check for the sample dataset
df = pd.read_csv('a.csv')

In [38]:
df=df[['x','class']]
df

,x,class
0,1,A
1,3,B
2,5,A
3,7,B
4,9,A
5,11,B
6,13,A


In [64]:
for attr in ["x"]:
    print('Interval for', attr)
    chimerge(data=df, attr=attr, label='class', max_intervals=5)

Interval for x
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
[1.33333333 0.66666667]
[0.66666667 0.33333333]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.75, 2.0, 2.0, 2.0, 2.0]
[ 1 , 5 ] freq -  3
[ 7 , 7 ] freq -  1
[ 9 , 9 ] freq -  1
[ 11 , 11 ] freq -  1
[ 13 , 13 ] freq -  1
